# Stable Diffusion WebUI (Automatic1111) for SageMaker Studio Lab

# Installation: Install ngrok reverse proxy
because creating a public gradio link doesn't work on SageMaker Studio Lab (at least not when I tried)

***You only need to do this once on your SageMaker SL account***

You will need a ngrok auth token later. If you don't have a ngrok account yet, create one here: https://dashboard.ngrok.com/signup

It's free for non-commercial use

In [ ]:
!mkdir -p ~/sagemaker-studiolab-notebooks/ngrok
%cd ~/sagemaker-studiolab-notebooks/ngrok
!wget https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.tgz -O ~/sagemaker-studiolab-notebooks/ngrok/ngrok.tgz
!tar zxf ngrok.tgz
!rm ngrok.tgz
!rm -rf ~/.config/ngrok
!mkdir -p ~/.config/ngrok
!echo 'version: "2"\nconsole_ui: false' >~/.config/ngrok/ngrok.yml 

# Installation: Clone webui repository
***You only need to do this once on your SageMaker SL account***

In [ ]:
%cd ~/sagemaker-studiolab-notebooks/
!git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui

# Installation: Download model(s) from huggingface
***You only need to do this once on your SageMaker SL account***

Alternatively you can use the JupyterLab file browser to upload one or more .ckpt files to the ~/sagemaker-studiolab-notebooks/stable-diffusion-webui/models/Stable-diffusion/ folder

#### Check free disk space

In [ ]:
!df -h | grep -E 'Avail|home'

#### Huggingface token

In [ ]:
import ipywidgets as widgets

token_textbox = widgets.Text(
    value='Enter Huggingface token here',
    description='Token:',
)
token_textbox

### Download model(s)
The next cell will download model ***v1-5-pruned-emaonly.ckpt*** (4GB) 

This model can be used for training textual inversion embeddings and hypernetworks

Comment out and uncomment lines in the next cell to download different models

Go to these pages and ***accept the licenses*** before running the next cell:

https://huggingface.co/runwayml/stable-diffusion-v1-5

https://huggingface.co/CompVis/stable-diffusion-v-1-4-original

https://huggingface.co/runwayml/stable-diffusion-inpainting

In [ ]:
user_header = f"\"Authorization: Bearer {token_textbox.value}\""
%cd ~/sagemaker-studiolab-notebooks/stable-diffusion-webui/models/Stable-diffusion/

# Model 1.4
#!wget --header={user_header} https://huggingface.co/CompVis/stable-diffusion-v-1-4-original/resolve/main/sd-v1-4.ckpt

# Model 1.5 (8GB) - only necessary for native training, not for embeddings/hypernetworks
#!wget --header={user_header} https://huggingface.co/runwayml/stable-diffusion-v1-5/resolve/main/v1-5-pruned.ckpt 

# Model 1.5 (4GB)
!wget --header={user_header} https://huggingface.co/runwayml/stable-diffusion-v1-5/resolve/main/v1-5-pruned-emaonly.ckpt 

# Inpainting model 1.5 (4GB)
#!wget --header={user_header} https://huggingface.co/runwayml/stable-diffusion-inpainting/resolve/main/sd-v1-5-inpainting.ckpt

### Download variational autoencoder (VAE)

Doing so can improve the quality of generated images, if you select the VAE in the webui settings

Go to these pages and ***accept the licenses*** before running the next cell:

https://huggingface.co/stabilityai/sd-vae-ft-ema-original

https://huggingface.co/stabilityai/sd-vae-ft-mse-original

In [ ]:
%cd ~/sagemaker-studiolab-notebooks/stable-diffusion-webui/models/VAE/

# sd-vae-ft-ema-original
#!wget --header={user_header} https://huggingface.co/stabilityai/sd-vae-ft-ema-original/blob/main/vae-ft-ema-560000-ema-pruned.ckpt

# sd-vae-ft-mse-original
!wget --header={user_header} https://huggingface.co/stabilityai/sd-vae-ft-mse-original/blob/main/vae-ft-mse-840000-ema-pruned.ckpt

# Installation: Uninstall opencv-python to avoid error when starting webui

***You only need to do this once on your SageMaker SL account***

In [ ]:
%pip uninstall opencv-python -y
#%pip install opencv-python-headless

In [ ]:
#%pip uninstall opencv-python-headless 

# Setup: Add ngrok auth token

***You only need to do this once***, unless you change your ngrok auth token

In [ ]:
import ipywidgets as widgets

ngrok_token_textbox = widgets.Text(
    value='Enter ngrok auth token here',
    description='Token:',
)
ngrok_token_textbox

In [ ]:
ngrok_auth_token = ngrok_token_textbox.value
!~/sagemaker-studiolab-notebooks/ngrok/ngrok config add-authtoken $ngrok_auth_token

# Run: Launch ngrok before launching webui


In [ ]:
# start ngrok as background process
import multiprocessing

def run_ngrok_bg():
    !~/sagemaker-studiolab-notebooks/ngrok/ngrok tcp 7860 --log=stdout > ~/sagemaker-studiolab-notebooks/ngrok/ngrok.log
    
multiprocessing.Process(target=run_ngrok_bg).start()


### Run next cell to see the public ngrok URL, replace tcp:// with http://

In [ ]:
!tail ~/sagemaker-studiolab-notebooks/ngrok/ngrok.log

# Installation: Install webui requirements.txt
I don't think this is necessary, but it doesn't hurt

In [ ]:
%cd ~/sagemaker-studiolab-notebooks/stable-diffusion-webui
!COMMANDLINE_ARGS="--exit" REQS_FILE="requirements.txt" python launch.py

# Run: Change into Web UI directory and download updates
This is not strictly necessary

In [ ]:
%cd ~/sagemaker-studiolab-notebooks/stable-diffusion-webui
!git pull

# Run: Launch web ui

Commandline arguments are:
  - `--share` - create online gradio.app link (this doesn't work on SageMaker SL)
  - `--gradio-debug` - print outputs to console
  - `--gradio-auth me:qwerty` - add authentication to gradio: username me, password qwerty
  
Change username and password in the next cell for ***security reasons***

In [ ]:
##################################

WEBUI_USERNAME="genius"
WEBUI_PASSWORD="artist"

##################################

ARGS = "\"--gradio-debug --gradio-auth " + WEBUI_USERNAME + ":" + WEBUI_PASSWORD + "\""

!echo $ARGS

%cd ~/sagemaker-studiolab-notebooks/stable-diffusion-webui
!COMMANDLINE_ARGS=$ARGS REQS_FILE="requirements.txt" python launch.py


End: Commands for ***after*** you have gotten done with a session
============================================================================

### Create .tar.gz archive of images for downloading 

Location will be ***~/sagemaker-studiolab-notebooks/outputs.tar.gz***

In [ ]:
%cd ~/sagemaker-studiolab-notebooks/stable-diffusion-webui/
!echo "Creating archive. Please wait..."
!tar -czf ~/sagemaker-studiolab-notebooks/outputs.tar.gz outputs
!echo "You can now delete the generated images and download the archive"

### Delete generated images from outputs folder

In [ ]:
!rm -rf ~/sagemaker-studiolab-notebooks/stable-diffusion-webui/outputs/*

### Restart kernel and delete logfile (this kills webui and ngrok)

In [ ]:
!rm ~/sagemaker-studiolab-notebooks/ngrok/ngrok.log
exit()